In [3]:
import tensorflow as tf

In [6]:
tf.__version__

'1.13.1'

In [26]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot
import pandas as pd

In [9]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
df.dropna(inplace=True)

In [15]:
df.shape

(18285, 5)

In [20]:
x=df.drop('label',axis=1)
y=df['label']

In [22]:
x.shape

(18285, 4)

In [23]:
y.shape

(18285,)

In [57]:
import nltk
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout

In [28]:
voc_size=5000

# One Hot Representation

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
messages=x.copy()
messages.reset_index(inplace=True)
ps=PorterStemmer()
corpus=[]
for i in range(len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

In [44]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [43]:
one_hot_rep=[one_hot(words,voc_size) for words in corpus]
one_hot_rep

[[4984, 1937, 4147, 3652, 1774, 3912, 4461, 1985, 2405, 2951],
 [3308, 2435, 2581, 4617, 2455, 1023, 3632],
 [2399, 2596, 3713, 4118],
 [581, 3891, 4504, 4639, 3226, 2343],
 [4441, 2455, 2674, 644, 1569, 2332, 2455, 552, 4609, 4838],
 [3397,
  700,
  4759,
  4238,
  4915,
  2868,
  776,
  3710,
  4691,
  1070,
  4315,
  992,
  3550,
  2766,
  3632],
 [2983, 4577, 1132, 646, 3202, 3473, 4815, 3047, 3783, 4050, 2136],
 [166, 151, 3260, 4085, 4263, 2258, 2868, 1731, 3783, 4050, 2136],
 [205, 4068, 43, 4800, 3020, 2730, 1303, 103, 2868, 2113],
 [1529, 4081, 2986, 3430, 4591, 3042, 2704, 782],
 [3860, 3856, 3463, 3057, 4839, 4832, 4529, 3396, 2613, 1311, 3507],
 [4639, 4202, 1774, 2730, 2868, 4263],
 [3347, 296, 4967, 1699, 649, 655, 1009, 3178, 1473],
 [1748, 3572, 2855, 2840, 706, 2726, 4385, 3783, 4050, 2136],
 [4349, 1436, 3931, 4241, 2151, 3783, 4050, 2136],
 [675, 2107, 2718, 3390, 102, 3633, 18, 4539, 814, 473],
 [1996, 4603, 2435],
 [76, 4938, 2161, 1006, 2868, 4160, 4485, 3632],
 [

# Word Embedding

In [45]:
sent_len=20
embeded_text=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_len)
print(embeded_text)

[[   0    0    0 ... 1985 2405 2951]
 [   0    0    0 ... 2455 1023 3632]
 [   0    0    0 ... 2596 3713 4118]
 ...
 [   0    0    0 ... 3783 4050 2136]
 [   0    0    0 ... 4592  364 4527]
 [   0    0    0 ... 2440  616 1957]]


Creating the Model

In [58]:
embedding_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_features,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [59]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
len(embeded_text),y.shape

(18285, (18285,))

In [61]:
import numpy as np
x_final=np.array(embeded_text)
y_final=np.array(y)

In [62]:
x_final.shape

(18285, 20)

In [63]:
y_final.shape

(18285,)

In [64]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.3, random_state=25)

# Traing the Model

In [66]:
model.fit(x_train,y_train,epochs=10, batch_size=64)

Epoch 1/10
12799/12799 [==============================] - 18s 1ms/sample - loss: 0.2226 - acc: 0.90716s - loss: 0.23 - ETA: 4s - loss: 0.2339 
Epoch 2/10
12799/12799 [==============================] - 19s 1ms/sample - loss: 0.1351 - acc: 0.9474
Epoch 3/10
12799/12799 [==============================] - 19s 1ms/sample - loss: 0.0999 - acc: 0.96353s - loss: 0.0942  - ETA: 2s 
Epoch 4/10
12799/12799 [==============================] - 19s 1ms/sample - loss: 0.0741 - acc: 0.97371s - loss: 0.
Epoch 5/10
12799/12799 [==============================] - 19s 1ms/sample - loss: 0.0553 - acc: 0.98096s - loss: 0.0549 - acc - ETA: 5s - lo
Epoch 6/10
12799/12799 [==============================] - 18s 1ms/sample - loss: 0.0383 - acc: 0.9876 - ETA: 15s - loss: 0.0388 - acc:  -  - ETA: 2s 
Epoch 7/10
12799/12799 [==============================] - 19s 2ms/sample - loss: 0.0320 - acc: 0.989314s - loss:  - E - ETA: 9s - loss: 0.02 - ETA: 8s - los - ETA: 2s - loss: 0.0315 - acc: 0.989 - ETA: 2s - loss
Epoch 8

In [72]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred=model.predict_classes(x_test)
cf=confusion_matrix(y_pred,y_test)
print('Confusion Matrics:\n\n',cf,'\n\n\n\n')
ac=accuracy_score(y_pred,y_test)
print('Accuracy Score:',ac)

Confusion Matrics:

 [[2825  276]
 [ 235 2150]] 




Accuracy Score: 0.9068538096974116
